In [1]:
import pandas as pd
import yfinance as yf

fiis = pd.read_csv('all-fiis.csv',header=None)
fiis['ticker']=fiis[1].str.split(';').str[1]
fiis = fiis[~fiis['ticker'].isnull()]

tickers = list(fiis['ticker'])

In [2]:
import time
import numpy as np

######
###### getting data for list of tickers
######

price_to_book = []
avg_div = []
price = []
price_med = []
price_q95=[]

for t in tickers:
    msft = yf.Ticker(t+".SA")

    ######### dividends history
    try:
        b = msft.history(period='1y',interval='1mo')
        b['Dividends'] = b['Dividends'].replace(0, float('nan'))
        avg = b['Dividends'].mean(skipna=True)
    except:
        avg = None
    avg_div.append(avg)


    ######## median and q95 historic
    try:
        # get all stock info
        h = msft.history(period='1y',interval='1mo')
        h['Close'] = h['Close'].replace(0, float('nan'))

        med = h['Close'].median(skipna=True)
        q95 = h['Close'].quantile(0.95)
        p = h.iloc[-1,:]['Close']
    except:
        med = None
        q95 = None
        p = None
    price_med.append(med)
    price_q95.append(q95)
    price.append(p)

    print('passed:',t)
    time.sleep(0.05)

df = pd.DataFrame(
    {'ticker': tickers,
     'price': price,
     'avg_div':avg_div,
     'median_price':price_med,
     'price_q95':price_q95
    })

df.head()

passed: KNIP11
passed: KNCR11
passed: HGLG11
passed: KNRI11
passed: IRDM11
passed: XPLG11
passed: MXRF11
passed: XPML11
passed: CPTS11
passed: HGRU11
passed: RECR11
passed: VISC11
passed: BTLG11
passed: HGBS11
passed: KNHY11
passed: HFOF11
passed: BCFF11
passed: BRCR11
passed: BRCO11
passed: TGAR11
passed: HGRE11
passed: JSRE11
passed: HGCR11
passed: VILG11
passed: MCCI11
passed: BBPO11
passed: HSML11
passed: VRTA11
passed: LVBI11
passed: TRXF11
passed: VCJR11
passed: RBVA11
passed: RBRR11
passed: HCTR11
passed: PVBI11
passed: HSLG11
passed: URPR11
passed: KNSC11
passed: RBRF11
passed: MALL11
passed: VGIP11
passed: ALZR11
passed: RZTR11
passed: CVBI11
passed: GTWR11
passed: GGRC11


ERROR:yfinance:FEXC11.SA: No data found, symbol may be delisted
ERROR:yfinance:FEXC11.SA: No data found, symbol may be delisted


passed: FEXC11
passed: VGIR11
passed: VGHF11
passed: BTCI11
passed: RZAK11
passed: DEVA11
passed: HABT11
passed: VINO11
passed: XPCI11
passed: RBRP11
passed: BTCR11
passed: SDIL11
passed: RBRL11
passed: MGFF11
passed: XPIN11
passed: SARE11
passed: RCRB11
passed: RBRY11
passed: BTAL11
passed: GALG11
passed: BCRI11
passed: RECT11
passed: KFOF11
passed: SNCI11
passed: BARI11
passed: CACR11
passed: MFII11
passed: BCIA11
passed: KISU11
passed: TEPP11


ERROR:yfinance:ARCT11.SA: No data found, symbol may be delisted
ERROR:yfinance:ARCT11.SA: No data found, symbol may be delisted


passed: ARCT11
passed: AFHI11
passed: XPSF11
passed: PATL11
passed: KCRE11
passed: FIIB11
passed: AIEC11
passed: PORD11
passed: RZAT11
passed: OUJP11
passed: BPFF11
passed: MCHF11
passed: LGCP11
passed: BLMG11
passed: SNFF11
passed: HGFF11
passed: BTRA11
passed: QAGR11
passed: NSLU11
passed: WHGR11
passed: RBFF11
passed: VIUR11
passed: HSAF11
passed: VTLT11
passed: CPFF11
passed: PLCR11
passed: BLMR11
passed: MORE11
passed: XPPR11
passed: RBHG11
passed: VCRI11
passed: JSAF11
passed: CYCR11
passed: NCHB11
passed: MORC11
passed: SADI11
passed: VSLH11
passed: ARRI11
passed: TORD11
passed: RVBI11
passed: HOFC11
passed: JPPA11
passed: SPTW11
passed: IBCR11
passed: CARE11
passed: RELG11
passed: VIFI11


,ticker,price,avg_div,median_price,price_q95
0,KNIP11,91.169998,0.793333,89.648788,91.604315
1,KNCR11,100.629997,1.139167,90.991371,100.656560
2,HGLG11,160.000000,1.216944,155.831207,160.193320
3,KNRI11,158.000000,1.002500,153.506439,159.697681
4,IRDM11,73.690002,0.841763,78.704376,83.215955


In [3]:
### calculate dividend yield
df['div_yield_hist'] = df['avg_div']/df['price'] * 100

### calculate price over median
df['price_to_median'] = df['price']/df['median_price']

### calculate price over max
df['price_to_max'] = df['price']/df['price_q95']

df.head()

,ticker,price,avg_div,median_price,price_q95,div_yield_hist,price_to_median,price_to_max
0,KNIP11,91.169998,0.793333,89.648788,91.604315,0.870169,1.016969,0.995259
1,KNCR11,100.629997,1.139167,90.991371,100.656560,1.132035,1.105929,0.999736
2,HGLG11,160.000000,1.216944,155.831207,160.193320,0.760590,1.026752,0.998793
3,KNRI11,158.000000,1.002500,153.506439,159.697681,0.634494,1.029273,0.989369
4,IRDM11,73.690002,0.841763,78.704376,83.215955,1.142303,0.936289,0.885527


In [4]:
def convert_to_float(value_str):
    if value_str.endswith('M'):
        multiplier = 1e6
        value_str = value_str.rstrip('M')
    elif value_str.endswith('B'):
        multiplier = 1e9
        value_str = value_str.rstrip('B')
    else:
        raise ValueError("Invalid format:",value_str)

    try:
        value = float(value_str.replace(',', ''))
        return value * multiplier
    except ValueError:
        raise ValueError("Invalid number format. Please provide a valid number.")

In [5]:
#####
#####  webscraping for price to book
#####
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

price_to_book = []
patrimonio=[]
yields=[]

for tic in tickers:
    url = "https://fiis.com.br/"+tic.lower()+"/"

    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    result = requests.get(url, headers=headers)
    soup = BeautifulSoup(result.text, "html.parser")
    print(result,tic)  # check if it worked

    indicators = soup.findAll(attrs = {'class':'indicators__box'})

    # Loop through the indicators to find the one with "P/VP"
    for indicator in indicators:
        p_element = indicator.findAll("p")
        if 'P/VP' in p_element[1]:  # Check if "P/VP" is present in the text of the <p> tag
            pvp = float(p_element[0].find("b").get_text().replace(",", "."))  # Extract the value and convert to float
            break
    price_to_book.append(pvp)

    # Loop through the indicators to find the one of patrimonio
    for indicator in indicators:
        p_element = indicator.findAll("p")
        if 'Patrimônio Líquido' in p_element[1]:  # Check if "Patrimônio Líquido" is present in the text of the <p> tag
            pat = convert_to_float(p_element[0].find("b").get_text())
            break
    patrimonio.append(pat)

    # loop to find dividend yield
    for indicator in indicators:
      p_element = indicator.findAll("p")
      #print(p_element[1])
      if 'Dividend Yield' in p_element[1]:  # Check if "P/VP" is present in the text of the <p> tag
          div = float(p_element[0].find("b").get_text().replace(",", "."))  # Extract the value and convert to float
          break
    yields.append(div/100)


<Response [200]> KNIP11
<Response [200]> KNCR11
<Response [200]> HGLG11
<Response [200]> KNRI11
<Response [200]> IRDM11
<Response [200]> XPLG11
<Response [200]> MXRF11
<Response [200]> XPML11
<Response [200]> CPTS11
<Response [200]> HGRU11
<Response [200]> RECR11
<Response [200]> VISC11
<Response [200]> BTLG11
<Response [200]> HGBS11
<Response [200]> KNHY11
<Response [200]> HFOF11
<Response [200]> BCFF11
<Response [200]> BRCR11
<Response [200]> BRCO11
<Response [200]> TGAR11
<Response [200]> HGRE11
<Response [200]> JSRE11
<Response [200]> HGCR11
<Response [200]> VILG11
<Response [200]> MCCI11
<Response [200]> BBPO11
<Response [200]> HSML11
<Response [200]> VRTA11
<Response [200]> LVBI11
<Response [200]> TRXF11
<Response [200]> VCJR11
<Response [200]> RBVA11
<Response [200]> RBRR11
<Response [200]> HCTR11
<Response [200]> PVBI11
<Response [200]> HSLG11
<Response [200]> URPR11
<Response [200]> KNSC11
<Response [200]> RBRF11
<Response [200]> MALL11
<Response [200]> VGIP11
<Response [200]>

In [6]:
##### gathering bases
df['price_to_book'] = price_to_book
df['div_yield'] = yields
df['patrimonio_mi'] = [pat / 1000000 for pat in patrimonio]

### check correlations
# Calculate the correlation matrix
correlation_matrix = df[['price_to_median','price_to_max','price_to_book']].corr()
# Now you can view the correlation between columns
print(correlation_matrix)

### filtering NAs
df.dropna(subset=["div_yield", "price_to_book"], how="all", inplace=True)
current_data = df.copy()

                 price_to_median  price_to_max  price_to_book
price_to_median         1.000000      0.706376       0.271516
price_to_max            0.706376      1.000000       0.735202
price_to_book           0.271516      0.735202       1.000000


# Reading predictions

In [7]:
import pickle

with open('future_values.pkl', 'rb') as file:
    future_values = pickle.load(file)

df_model = pd.read_csv('df_model.csv')

df = current_data.merge(df_model,
                          how='left',
                          left_on='ticker',
                          right_on='Predicted_FIIs')
df.drop(columns=['Predicted_FIIs','Y_Test','Y_Pred'],inplace=True)

df.head()

,ticker,price,avg_div,median_price,price_q95,div_yield_hist,price_to_median,price_to_max,price_to_book,div_yield,patrimonio_mi,Mean_of_Future_Values_3_mo,Max_of_Future_Values_3_mo,Value_2023_11_01
0,KNIP11,91.169998,0.793333,89.648788,91.604315,0.870169,1.016969,0.995259,0.95,0.1066,77000.0,90.669380,91.251263,91.280075
1,KNCR11,100.629997,1.139167,90.991371,100.656560,1.132035,1.105929,0.999736,1.00,0.1350,58000.0,102.345128,103.588613,100.696404
2,HGLG11,160.000000,1.216944,155.831207,160.193320,0.760590,1.026752,0.998793,1.04,0.0919,52000.0,158.325035,159.390777,160.276321
3,KNRI11,158.000000,1.002500,153.506439,159.697681,0.634494,1.029273,0.989369,0.99,0.0767,38000.0,155.169214,155.818393,158.501129
4,IRDM11,73.690002,0.841763,78.704376,83.215955,1.142303,0.936289,0.885527,0.85,0.1371,32000.0,71.779485,73.613316,71.778496


# Rank formula zero:

rank_0 =

(1+3*div_hist/12) *

Mean_of_Future_Values_3_mo/price *

1/price_to_book


In [8]:
final = df[['ticker',
            'price',
            'price_q95',
            'patrimonio_mi',
            'div_yield_hist',
            'div_yield',
            'price_to_book',
            'Mean_of_Future_Values_3_mo',
            'Max_of_Future_Values_3_mo']].copy()

# final['p1'] = np.where(
#     final['div_yield_hist'].isna(),
#     final['div_yield'],
#     (final['div_yield_hist'] + final['div_yield']) / 2
# )

final['p1'] = 1.0+(3.0*final['div_yield']/12.0)

final['p2'] = np.where(
    final['Max_of_Future_Values_3_mo'].isnull(),
    final['price_q95'],
    final['Max_of_Future_Values_3_mo']
)
final['p2'] = final['p2']/final['price']

final['rank_0'] = final['p1'] * final['p2'] / final['price_to_book']

## dropping columns and nan rows
final.drop(columns=['price_q95','p1','p2'],inplace=True)
final = final[~final['rank_0'].isna() & ~np.isinf(final['rank_0'])]

## priting
final.sort_values(by='rank_0',ascending=False).head(10)

,ticker,price,patrimonio_mi,div_yield_hist,div_yield,price_to_book,Mean_of_Future_Values_3_mo,Max_of_Future_Values_3_mo,rank_0
37,KNSC11,8.800000,12000.0,9.261363,0.1188,0.10,9.804764,10.386211,12.153046
16,BCFF11,8.970000,20000.0,6.243032,0.0936,0.11,8.826931,8.917299,9.248975
120,CARE11,1.410000,2968.0,NaN,0.0000,0.17,1.404557,1.851824,7.725591
116,HOFC11,19.680000,3382.0,0.940041,0.1032,0.22,NaN,NaN,5.549245
104,XPPR11,18.320000,5088.0,0.818777,0.0873,0.26,13.252456,16.734569,3.589982
33,HCTR11,36.060001,25000.0,1.659387,0.1991,0.32,21.033436,35.632671,3.241671
112,VSLH11,4.070000,3110.0,0.772201,0.1474,0.39,3.726913,3.839863,2.508261
121,RELG11,49.529999,1660.0,1.413285,0.0606,0.40,45.770879,48.450783,2.482577
92,BTRA11,59.990002,4238.0,1.116853,0.1479,0.48,59.521242,60.496895,2.178619
51,DEVA11,44.660000,14000.0,1.305099,0.1955,0.46,30.283561,40.991347,2.092856


# Creating the size column

In [9]:
# Define the quantiles and labels
quantiles = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
labels = ['A', 'AA', 'AAA', 'AAAA', 'AAAAA']

# Create the 'size' column based on quantiles
final['size'] = pd.qcut(final['patrimonio_mi'], q=quantiles, labels=labels)

## filtering only big ones
a = final[final['size'].isin(['AAAA', 'AAAAA'])]

a.sort_values(by='rank_0',ascending=False).head(20)

,ticker,price,patrimonio_mi,div_yield_hist,div_yield,price_to_book,Mean_of_Future_Values_3_mo,Max_of_Future_Values_3_mo,rank_0,size
37,KNSC11,8.800000,12000.0,9.261363,0.1188,0.10,9.804764,10.386211,12.153046,AAAA
16,BCFF11,8.970000,20000.0,6.243032,0.0936,0.11,8.826931,8.917299,9.248975,AAAAA
33,HCTR11,36.060001,25000.0,1.659387,0.1991,0.32,21.033436,35.632671,3.241671,AAAAA
51,DEVA11,44.660000,14000.0,1.305099,0.1955,0.46,30.283561,40.991347,2.092856,AAAA
17,BRCR11,59.290001,27000.0,0.757576,0.0909,0.59,54.678492,55.621786,1.626186,AAAAA
21,JSRE11,71.639999,23000.0,0.652568,0.0779,0.64,68.269646,69.708100,1.549974,AAAAA
36,URPR11,82.123672,12000.0,1.489043,0.1781,0.82,82.261898,84.550625,1.311455,AAAA
4,IRDM11,73.690002,32000.0,1.142303,0.1371,0.85,71.779485,73.613316,1.215528,AAAAA
10,RECR11,84.010002,25000.0,1.032215,0.1285,0.89,84.267049,85.758222,1.183824,AAAAA
32,RBRR11,87.000000,14000.0,0.953450,0.1144,0.93,89.186027,90.392266,1.149147,AAAA


In [10]:
## saving final file

# Specify the file path where you want to save the CSV file
csv_file_path = 'final_data.csv'

final.to_csv(csv_file_path, index=False)  # Set index=False to exclude the index column in the CSV

# Trigger a download of the CSV file
from google.colab import files
files.download(csv_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>